## Solution for $\mathbb{H}_{u,v}$

In this notebook we validate our recursive solution to the integral $\mathbb{H}$.

In [4]:
import sympy
from sympy import *
import numpy as np
np.random.seed(0)

In [5]:
# Initialize the session
init_session(quiet=True)
print("Using sympy version", sympy.__version__)

# Define our symbols
phi, lam1, lam2 = symbols('phi lambda_1 lambda_2')
lam = Array((lam1, lam2))


Using sympy version 1.4


Here's the exact version of the integral, computed with `sympy`:

In [6]:
def Hexact(u, v):
    return integrate(cos(phi) ** u * sin(phi) ** v, (phi, lam1, lam2))

Here's our recursive version:

In [7]:
def Delta(x):
    return x[1] - x[0]


def H(u, v):
    if u == v == 0:
        return Delta(lam)
    elif u == 1 and v == 0:
        return Delta(lam.applyfunc(sin))
    elif u == 0 and v == 1:
        return Delta(lam.applyfunc(lambda x: -cos(x)))
    elif u == 1 and v == 1:
        return Delta(lam.applyfunc(lambda x: -cos(x) ** 2 / 2))
    elif u < 2 and v >= 2:
        return (
            Delta(lam.applyfunc(lambda x: -cos(x) ** (u + 1) * sin(x) ** (v - 1)))
            + (v - 1) * H(u, v - 2)
        ) / (u + v)
    else:
        return (
            Delta(lam.applyfunc(lambda x: cos(x) ** (u - 1) * sin(x) ** (v + 1)))
            + (u - 1) * H(u - 2, v)
        ) / (u + v)

`sympy` has a little trouble checking for the analytical equivalence of the expressions as `u` and `v` get large, so let's check that the expressions agree numerically for several different values of $\lambda$:

In [9]:
for i in range(10):
    x1, x2 = np.random.random(2) * 2 * np.pi
    for u in range(5):
        for v in range(5):
            diff = (H(u, v) - Hexact(u, v)).replace(lam1, x1).replace(lam2, x2)
            assert abs(diff) < 1e-15